In [1]:
import json
from dependencies.handle_gre_data import HandleGREData

# Define configuration
config = {
    'traders': ['Beimnet', 'Bouchra', 'Eduardo', 'Felman'],
    'strategies': {
        'BTC': ['Core BTC', 'Options $', 'RelativeValue BTC'],
        'ETH': ['Core ETH', 'Options $', 'RelativeValue ETH'],
        'SOL': ['Core SOL', 'Options $', 'RelativeValue SOL'],
        'Alts': ['Core', 'Options', 'RelativeValue']
    }
}

# Create a DeltaProcessor instance
processor = HandleGREData(config)

# Process the CSV file
csv_filepath = '../data/GDLP GRE Data 23072024.csv'
final_df = processor.process_csv(csv_filepath)

# Save the results
final_df.to_csv('../output/23072024.csv', index=False)
final_df.to_excel('../output/23072024.xlsx', index=False)
print(final_df)


        Beimnet BTC Core BTC Delta  Beimnet BTC Options $ Delta  \
Values                1.134822e+07                 1.461724e+07   

        Beimnet BTC Delta  Beimnet ETH Core ETH Delta  Beimnet ETH Delta  \
Values       2.596546e+07               -1.039035e+07      -1.039035e+07   

        Beimnet Delta  Bouchra BTC Core BTC Delta  \
Values   1.557511e+07                1.178070e+07   

        Bouchra BTC Options $ Delta  Bouchra BTC Delta  \
Values                 7.090302e+06       1.887100e+07   

        Bouchra ETH Core ETH Delta  ...  GDLP ex-Novo ETH Delta  \
Values                2.356219e+06  ...            3.935608e+08   

        GDLP ex-Novo SOL Delta  GDLP ex-Novo Alts Delta  GDLP ex-Novo Delta  \
Values           -9.940977e+07             1.189462e+09        6.412737e+09   

        GDLP ex-Novo ex-Alts Delta  GDLP BTC Total Delta  \
Values                5.223275e+09          2.102558e+09   

        GDLP ETH Total Delta  GDLP SOL Total Delta  GDLP Alts Total Delta